<a href="https://colab.research.google.com/github/JJ-Pineda/simple_ai_demos/blob/main/qwen3_1_7b_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install bitsandbytes==0.47.0

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True
)

qwen = AutoModelForCausalLM.from_pretrained(
    'Qwen/Qwen3-1.7B',
    torch_dtype=None,
    quantization_config=bnb_config,
    device_map='cuda'
)

tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-1.7B')


In [1]:
my_message = 'Why is the meaning of life?'

input_ids = tokenizer.apply_chat_template(
    [[{'role': 'user', 'content': my_message}]],
    add_generation_prompt=True,
    enable_thinking=False,
    return_tensors='pt'
).to('cuda')

results = qwen.generate(
    input_ids,
    attention_mask=input_ids == input_ids,
    max_new_tokens=200,
    temperature=0.6,
    top_p=0.95,
    top_k=20
)

response = results[:, input_ids.shape[1]:]

print(tokenizer.batch_decode(response)[0])
